## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        print(mean, std)
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)
120.70756512369792 64.1500758911213


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train_onehot = one_hot.fit_transform(y_train).toarray()
y_test_onehot = one_hot.transform(y_test).toarray()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''


#卷積組合
classifier.add(Convolution2D(32, (3,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu


#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train_onehot, batch_size=100, epochs=40)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/40
50000/50000 [==============================] - 15s 295us/step - loss: 1.3193 - accuracy: 0.5394
Epoch 2/40
50000/50000 [==============================] - 12s 243us/step - loss: 0.9349 - accuracy: 0.6744
Epoch 3/40
50000/50000 [==============================] - 13s 267us/step - loss: 0.7847 - accuracy: 0.7268
Epoch 4/40
50000/50000 [==============================] - 13s 252us/step - loss: 0.6857 - accuracy: 0.7596
Epoch 5/40
50000/50000 [==============================] - 13s 254us/step - loss: 0.6033 - accuracy: 0.7904
Epoch 6/40
50000/50000 [==============================] - 13s 269us/step - loss: 0.5359 - accuracy: 0.8111
Epoch 7/40
50000/50000 [==============================] - 12s 239us/step - loss: 0.4711 - accuracy: 0.8336
Epoch 8/40
50000/50000 [==============================] - 11s 228us/step - loss: 0.4111 - accuracy: 0.8564
Epoch 9/40
50000/50000 [==============================] - 14s 271us/step - loss: 0.3671 - accuracy: 0.8727
Epoch 10/40
50000/50000 [============

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example = (np.zeros(shape=(1,32,32,3))-mean_train) / (std_train+1e-7) 
classifier.predict(input_example)

array([[4.8026158e-07, 8.5323942e-10, 9.4119437e-02, 8.1266707e-01,
        7.8729779e-02, 1.3209229e-03, 7.5897167e-04, 1.2394316e-02,
        8.9646537e-06, 2.2237071e-14]], dtype=float32)

In [5]:
classifier.evaluate(x_test, y_test_onehot)

10000/10000 [==============================] - 10s 980us/step


[2.2728396960258483, 0.6704000234603882]

In [27]:
# y_inv = one_hot.inverse_transform(y_pred)
classifier.evaluate(x_test, y_test_onehot)

10000/10000 [==============================] - 2s 153us/step


[4.409608975219727, 0.6776000261306763]